# DF: Assignment 2: Database forensics

In [1]:
from preprocessing import attack_df

In [2]:
attack_df

,id,processid,victim,port_service`_type,type,duration,time,stopped,client_attacker,server_used_in_th_attack
0,1,5672,91.214.20.52,80,ESSYN,33,1358989954,1,1,94.102.49.76
1,2,29065,91.214.20.52,80,UDP,1783,1358990329,1,1,93.174.93.30
2,3,23005,78.31.8.124,80,ESSYN,393,1358991360,0,1,89.248.172.97
3,4,29411,199.204.44.188,80,ESSYN,60,1358995416,0,4,93.174.93.30
4,5,19298,chrome-mods.com,80,RUDY,50000,1358995471,1,3,72.9.144.80
...,...,...,...,...,...,...,...,...,...,...
48768,48840,15874|1992|28878|32327,109.234.152.119,1973,ESSYN,300,1363351101,0,258,94.102.49.79|94.102.51.252|89.248.160.146|89.2...
48769,48841,15022|19087|28887,94.242.205.227,80,SOURCE,65,1363351473,1,261,72.9.144.80|67.222.156.241|89.248.160.146
48770,48842,15026|19091|28896,94.242.205.227,80,ARME,65,1363351493,1,261,72.9.144.80|67.222.156.241|89.248.160.146
48771,48843,15898|3409|32693|2010,94.242.205.227,80,ESSYN,65,1363351507,1,261,94.102.49.79|89.248.172.96|89.248.162.242|94.1...
